In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression as LR
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.model_selection import cross_val_score, KFold

In [2]:
dataframe = pd.read_csv('salida/resultadoMRIT2.csv')

In [3]:
dataframe

,suma,distancias,entropias,mutual_info,m_earth,max_info_p,antonimos,relaciones,CLASS
0,-0.264991,294.031950,4.839,9.576863,0.037193,0.517302,0,1,entailment
1,0.103484,189.135700,2.585,1.784762,0.006869,0.251793,0,1,entailment
2,-0.009624,123.245390,3.948,3.902809,0.017327,0.289142,0,0,contradiction
3,0.000000,0.000000,0.000,0.000000,0.000000,NaN,0,0,contradiction
4,0.176506,113.819000,2.750,3.763009,0.011833,0.373197,0,0,neutral
...,...,...,...,...,...,...,...,...,...
95,-0.080292,247.618470,4.438,7.502547,0.035230,0.464188,0,1,entailment
96,0.000000,37.524600,0.000,0.000000,0.000000,NaN,0,0,entailment
97,0.160267,189.692380,3.640,5.416953,0.026852,0.316492,0,1,contradiction
98,0.291853,97.558205,2.000,3.773990,0.010269,0.363122,0,1,contradiction


In [4]:
X = np.array(dataframe.drop(['CLASS'],axis=1))
y_ = np.array(dataframe['CLASS'])

In [5]:
y=[]
for i in y_:
    if i == "contradiction":
        y.append(0)
    elif i == "entailment":
        y.append(1)
    else:
        y.append(2)

In [6]:
scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.transform(X)
X = X_scaled

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=7)

In [8]:
X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

In [9]:
import xgboost as xgb

In [9]:
# Define hyperparameters
#params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}

In [10]:
clf_xgb = xgb.XGBClassifier(objective='reg:logistic',n_estimators=1000,device="cuda")
clf_xgb.fit(X_train, y_train)
preds_xgb = clf_xgb.predict(X_test)

/home/david/anaconda3/envs/rit/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [21:43:56] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


In [11]:
scores = cross_val_score(clf_xgb, X_train, y_train, cv=5)
print("Mean cross-validation score: %.2f" % scores.mean())

Mean cross-validation score: 0.50


In [12]:
kfold = KFold(n_splits=20, shuffle=True)
kf_cv_scores = cross_val_score(clf_xgb, X_train, y_train, cv=kfold )
print("K-fold CV average score: %.2f" % kf_cv_scores.mean())

K-fold CV average score: 0.53


In [13]:
accuracy_score(y_test,preds_xgb)

0.75

In [18]:
clf_lr = LR().fit(X_train, y_train)

In [19]:
accuracy_xgb = float(np.sum(preds_xgb == np.array(y_test)))/len(np.array(y_test))
print('Accuracy de XGBoost: ', accuracy_xgb)
accuracy_lr = clf_lr.score(X_test, y_test)
print('Accuracy de Logistic Regression: ', accuracy_lr)

Accuracy de XGBoost:  0.75
Accuracy de Logistic Regression:  0.7


In [20]:
confusion_matrix(y_test, preds_xgb)

array([[4, 0, 1],
       [1, 6, 0],
       [0, 3, 5]])

In [21]:
data=xgb.DMatrix(X, label=y)

In [16]:
#Performing cross-validation
#cv_results = xgb.cv(data, num_boost_round=10, nfold=3,metrics='logloss', seed=42)

#Printing the results
#print(cv_results)

TypeError: cv() missing 1 required positional argument: 'dtrain'